# Verification of Generated Alarm Data

Goal: Verify alarm data generated with `generate_alarm_data.py` and chunked resp. non-chunked cleaned CHARTEVENTS.

The cleaned CHARTEVENTS data set can be found on the server at `./data/chartevents_clean.parquet` and the chunked version can be found at `./data/chartevents_clean_values_and_thresholds_with_chunkid_65.parquet`.

## Compare overall number of alarms

In [ ]:
import pandas as pd

alarm_violations = pd.read_csv('../../data/alarms/alarm_violations.csv')
print("Original #Alarms:", len(alarm_violations))

alarm_data = pd.read_csv('../../data/alarms/alarm_data.csv')
print("#Alarms for Non-chunked Version:", len(alarm_data))

alarm_data_with_chunks = pd.read_csv('../../data/alarm_data_with_chunks_65.csv')
print("#Alarms for Chunked Version:", len(alarm_data_with_chunks))

The number of alarms generated with the cleaned data sets of CHARTEVENTS with three parameters is less (285,350) than the number reached with all CHARTEVENTS with five parameters (388,209), which makes sense. Additionally, the cleaned data set has the same number of alarms, regardless of whether it has been chunked.

## Investigate threshold types

In [ ]:
alarm_data_high = alarm_data[(alarm_data['THRESHOLD_TYPE'] == 'HIGH')]
print("HIGH Alarms (Cleaned):", len(alarm_data_high))

alarm_data_low = alarm_data[(alarm_data['THRESHOLD_TYPE'] == 'LOW')]
print("LOW Alarms (Cleaned):", len(alarm_data_low))

alarm_data_high_old = alarm_violations[(alarm_violations['THRESHOLD_TYPE'] == 'HIGH') & (alarm_violations['ITEMID'].isin([220045, 220179, 220277]))]
print("HIGH Alarms (Original):", len(alarm_data_high_old))

alarm_data_low_old = alarm_violations[(alarm_violations['THRESHOLD_TYPE'] == 'LOW') & (alarm_violations['ITEMID'].isin([220045, 220179, 220277]))]
print("LOW Alarms (Original):", len(alarm_data_low_old))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

alarms_by_itemid_type = alarm_data\
    .groupby(['ITEMID', 'THRESHOLD_TYPE'])\
    .size()\
    .reset_index(name='Count')
alarms_by_itemid_type = alarms_by_itemid_type.pivot(index='ITEMID', columns='THRESHOLD_TYPE', values='Count')

# define figure
sns.set_style('whitegrid')
fig, ax = plt.subplots(1, figsize=(16, 6))
# numerical x
x = np.arange(0, len(alarms_by_itemid_type.index))
# plot bars
plt.bar(x - 0.1, alarms_by_itemid_type['LOW'], width = 0.2, color = '#1D2F6F')
plt.bar(x + 0.1, alarms_by_itemid_type['HIGH'], width = 0.2, color = '#8390FA')

# x and y details
plt.xlabel('ITEM ID',fontsize=16)
plt.ylabel('Alarm Counts',fontsize=16)
plt.xticks(x, alarms_by_itemid_type.index)

# title and legend
plt.title('Alarm Counts by ITEM ID and TYPE', fontsize=18)
plt.legend(['LOW','HIGH'], loc='upper left', ncol = 2)

plt.show()

**HIGH alarms:** 155,664 -> 138,895

**LOW alarms:** 159,290 -> 146,455

The distribution of alarms per ITEMID and THRESHOLD_TYPE is comparable to the result with the original data set. Overall, both alarm types occur approximately equally often.

## Check if all ICU stays have same number of alarms

In [ ]:
unique_icustays = pd.read_parquet('../../data/unique_icustays_in_chartevents_subset.parquet', engine='pyarrow')

for icustay in unique_icustays.ICUSTAY_ID:
    alarm_data_of_icustay = alarm_data[alarm_data['ICUSTAY_ID'] == icustay]
    alarm_data_of_icustay_with_chunks = alarm_data_with_chunks[alarm_data_with_chunks['ICUSTAY_ID'] == icustay]

    if len(alarm_data_of_icustay) != len(alarm_data_of_icustay_with_chunks):
        print(f'ICUSTAY_ID {icustay} actually has {len(alarm_data_of_icustay)} alarms and {len(alarm_data_of_icustay_with_chunks)} alarms when chunked')

All ICU stays have the same number of alarms, regardless of whether the cleaned CHARTEVENTS have been chunked.

## Investigate alarms of single ICU stay

Investigated several ICU stays, see slide deck "Weekly_06".

In [ ]:
# Variables to adjust
selected_icustay = 299632
selected_param = 'Blood Pressure'
selected_param_itemids = [220179, 223752, 223751] # start with ITEMID of values
selected_param_unit = 'Millimeter Mercury (mmHg)'
with_chunked_alarms = False

In [ ]:
filename_suffix = ''

if with_chunked_alarms:
    alarm_data_of_stay = alarm_data_with_chunks[(alarm_data_with_chunks['ICUSTAY_ID'] == selected_icustay) & (alarm_data_with_chunks['ITEMID'] == selected_param_itemids[0])]
    filename_suffix = '_chunked'
else:
    alarm_data_of_stay = alarm_data[(alarm_data['ICUSTAY_ID'] == selected_icustay) & (alarm_data['ITEMID'] == selected_param_itemids[0])]

print(alarm_data_of_stay.CHARTTIME)

In [ ]:
chartevents = pd.read_parquet('../../data/chartevents_clean.parquet', engine='pyarrow')
chartevents_of_stay = chartevents[(chartevents['ICUSTAY_ID'] == selected_icustay) & (chartevents['ITEMID'].isin(selected_param_itemids))]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set variables for plot
title = f'History for Systolic {selected_param} of ICU Stay {selected_icustay}'
xlabel = 'Time'
ylabel = selected_param_unit
plotdata = chartevents_of_stay
xvalue = 'CHARTTIME'
yvalue = 'VALUENUM_CLEAN'
huevalue = 'ITEMID'

# Config figure
sns.set_style('whitegrid')
fig, ax = plt.subplots(
    figsize=(11, 5),
    dpi=72 # e.g. 72 for screen, 300 for print
)

# Plot lines for values and its thresholds
sns.lineplot(
    data=plotdata,
    x=xvalue,
    y=yvalue,
    hue=huevalue,
    style=huevalue,
    drawstyle='steps-post', # Interpolate missing values by using the last available value
    markers=['p','^','v'],
    markersize=5,
    dashes=False,
    palette=[sns.color_palette('colorblind')[1], sns.color_palette('colorblind')[2], sns.color_palette('colorblind')[0]])

# Plot dotted vertical lines for alarms
for alarm_time in alarm_data_of_stay.CHARTTIME:
    plt.axvline(
        x=pd.to_datetime(alarm_time),
        linestyle='dotted',
        color='r')

# Configure legend
plt.plot([], linestyle='dotted', color='r', label='Triggered Alarm')
plt.legend(title=None, bbox_to_anchor=(1.02, 0.3), loc='upper left', borderaxespad=0)

# Configure title and labels
ax.set_title(title, fontweight='bold', color='black', fontsize=14, y=1.05)
ax.set_xlabel(xlabel, fontsize=12, labelpad=15)
ax.set_ylabel(ylabel, fontsize=12, labelpad=15)
plt.xticks(rotation=45)

# Plot figure
plt.tight_layout()
#plt.savefig(f'../plots/verified_alarm_data_{selected_param.lower().replace(' ', '_')}_for_stay_{selected_icustay}{filename_suffix}.png', dpi=1200)
plt.show(fig)

For all tested ICU stays, on the one hand sensible plots and on the other hand the same plots could be observed for chunked and non-chunked versions of cleaned CHARTEVENTS.
